# *Movie Recommend System*

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv(r'/content/tmdb_5000_movies.csv',encoding='latin1')


# since the datset is not encoded in utf-8 so we have to specify it seperately

In [ ]:
data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",12/10/09,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",5/19/07,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from BondÛªs past sends him...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",10/26/15,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",7/16/12,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",3/7/12,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [ ]:
data.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [ ]:
# some features are in JSON format we have to change them.We want generes and keyword as it is most important to form relation between movies.

data['genres'].loc[0]


'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [ ]:
import json

x = json.loads(data['genres'][0])
x

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [ ]:
for i in x:
  print(i['name'])

Action
Adventure
Fantasy
Science Fiction


In [ ]:
' '.join(''.join(ele['name'].split()) for ele in x)

# we created string of generes,we do this for each row
# same process is done with geners column

'Action Adventure Fantasy ScienceFiction'

In [ ]:
def generes_and_keywords(row):
  genre = json.loads(row['genres'])
  genres = ' '.join(''.join(gen['name'].split()) for gen in genre)

  keyword = json.loads(row['keywords'])
  keywords = ' '.join(''.join(key['name'].split()) for key in keyword)

  return "%s %s" % (genres,keywords)

In [ ]:
data['string'] = data.apply(generes_and_keywords,axis=1)

# these are used to vectorize using TF-IDF,we then compare our movie title with these vectors and create similarity between them.

In [ ]:
data['string'][0]

'Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=1000)

In [ ]:
vectors = tfidf.fit_transform(data['string'])

In [ ]:
vectors

<4803x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 43974 stored elements in Compressed Sparse Row format>

*creating word to index and index to word mapping for title of movies*

In [ ]:
title = data['title']

In [ ]:
title

0                                         Avatar
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
                          ...                   
4798                                 El Mariachi
4799                                   Newlyweds
4800                   Signed, Sealed, Delivered
4801                            Shanghai Calling
4802                           My Date with Drew
Name: title, Length: 4803, dtype: object

In [ ]:
word2indx = {}
index = 0

for i in title:
  word2indx[i] = index
  index += 1

In [ ]:
idx = word2indx['Dark Water']

In [ ]:
# Dark Water movie corresponding row in our tfidf matrix

movie = vectors[idx]

***cosine_similarity***

*Cosine similarity measures the similarity between two vectors of an inner product space. It is measured by the cosine of the angle between two vectors and determines whether two vectors are pointing in roughly the same direction.*

*Cosine similarity is a value bound by a constrained range of 0 and 1.Suppose the angle between the two vectors were 90 degrees. In that case, the cosine similarity will have a value of 0 i.e., no similarity*

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
help(cosine_similarity)

In [ ]:
similarity = cosine_similarity(movie,vectors)

In [ ]:
similarity

array([[0.        , 0.        , 0.08368888, ..., 0.01247819, 0.        ,
        0.        ]])

In [ ]:
scores = similarity.flatten()

In [ ]:
recommend = scores.argsort()[::-1][:6]

# these index has top 5 similarity score for our movie

In [ ]:
# converting these index back to similar movies names

data['title'].iloc[recommend]

# Only one movie 'Stepmom' is not related rest are perfectly related to Dark Water which horror/thriller movie.

779                                           Dark Water
3220    The Haunting in Connecticut 2: Ghosts of Georgia
2533                                  The Woman in Black
2306                               The Amityville Horror
3558                                          Lights Out
915                                              Stepmom
Name: title, dtype: object

*function to recommend any movie related to our search*

In [ ]:
def recommender(movie_name):
  index = word2indx[movie_name]

  movie_tfidf = vectors[index]

  similar = cosine_similarity(movie_tfidf,vectors)

  score = similar.flatten()

  top_5_index = score.argsort()[::-1][:6]

  top_5_movies = data['title'].iloc[top_5_index]

  return top_5_movies

In [ ]:
recommender('The Dark Knight Rises')

3       The Dark Knight Rises
65            The Dark Knight
2864                Arbitrage
1740               Kick-Ass 2
119             Batman Begins
428            Batman Returns
Name: title, dtype: object